## Examples for running LOS Q,U and RM synthesis simulation routine

### Necessary imports:

In [ ]:
from qu_los_sim import make_los
import numpy as np
import matplotlib.pyplot as plt

### Description available in help:

In [ ]:
help(make_los)

### A plotting routine:

In [ ]:
def quick_plot(results):

    fig, ax = plt.subplots(2,1,figsize=(12,10))

    polangle = 0.5*np.arctan2(results['pol'].imag, results['pol'].real)
    
    ax[0].scatter(results['lsq'],abs(results['pol']), s=0.5)
    ax[0].scatter(results['lsq'],results['pol'].real, s=0.5)
    ax[0].scatter(results['lsq'],results['pol'].imag, s=0.5)
    
    ax2 = ax[0].twinx()
    ax2.scatter(results['lsq'], polangle, color='red', s=0.5)
    ax2.set_ylim(-np.pi/2,np.pi/2)

    ax[0].set_xlabel(r'$\lambda^2$ (m$^2$)')
    ax[0].set_ylabel(r'Polarised intensity')
    ax2.set_ylabel(r'Polarisation angle (rad)')
    
    ax[1].scatter(results['fdf_dirty'][1]['phiArr_radm2'], abs(results['fdf_dirty'][1]['dirtyFDF']),s=0.5)
    ax[1].grid()
    ax[1].set_xlabel(r'Faraday depth (rad m$^{-2}$)')
    ax[1].set_ylabel(r'Polarised intensity')

    return

### Overview:

For each LOS the model is the sum of any number of components (j) of the form:

$$p_j = p_{0,j} e^{2i(\psi_{0,j}+\phi_{0,j}\lambda^2)} \frac{\sin(\Delta \phi_j \lambda^2)}{\Delta \phi_j \lambda^2}e^{-2\sigma^2_j\lambda^4}$$

The following parameters must be defined in a dictionary:

 - $p_{0,j}$ = polarised intensity of the jth component in arbitrary units ("pi")
 - $\phi_{0,j}$ = Faraday depth of the jth (screen) component in rad/m$^2$ ("phi")
 - $\psi_{0,j}$ = Polarisation angle of the jth (screen) component in degrees ("psi")
 - $\sigma_j$ = Burn depolarization of the jth component in rad/m$^2$ ("sig")
 - $\Delta \phi_j$ = Width of jth component Burn slab in rad/m$^2$ ("dphi")

Set all non-required parameters to zero. For example, if only a Faraday screen is needed (no Burn slab), set "dphi" to zero.

If multiple components along the LOS or within the telescope beam are to be modeled, populate the dictionary with lists of parameters. Each item in the list corresponds to a component to be modeled, and all components will be summed in complex polarization.

Create a list of frequencies. This can be an equally spaced set of frequencies, unequally spaced, or with gaps to account for RFI, for example.

When running the code for your LOS, the parameters will be summarized and the type(s) of model(s) these correspond to will be indicated.

## (1) Examples with one Faraday depth screen

### Set up the parameters and frequency array

In [ ]:
parameters = {
              "pi"  : [1.0],  # PI of the one component
              "phi" : [50.0], # Faraday depth of 50
              "psi" : [10.0], # Polarization angle of 10 degrees
              "sig" : [0.0],  # No Burn depolarization
              "dphi": [0.0]   # No Burn slab
            }
freqs = np.arange(400e6,1800e6,0.5e6) # Frequencies covering 400 MHz to 1.8 GHz in steps of 0.5 MHz

### (a) No noise in data, uniform weights for RM synthesis:

In [ ]:
# Run the code with the above parameters and a value for the Gaussian noise, toggle RM synthesis on:
results = make_los(parameters, 
                   freqs=freqs, 
                   do_rmsynth=True)

# Display the output parameters available:
print(results.keys())

# Note: the RM synthesis results are stored in results['fdf_dirty'][1]:
print(results['fdf_dirty'][1].keys())

# Make some plots to summarize the results
quick_plot(results)

### (b) Gaussian random noise (with same standard deviation in all channels), uniform weights for RM synthesis:

In [ ]:
results = make_los(parameters, 
                   freqs=freqs, 
                   q_noise_in=0.8, 
                   u_noise_in=0.8, 
                   do_rmsynth=True)

quick_plot(results)


### (c) Frequency-dependent Gaussian random noise, uniform weights for RM synthesis:

In [ ]:
# Make array of q and u standard deviations for frequency-dependent noise
q_noise_in = np.linspace(0.8,0.01,len(freqs))
u_noise_in = np.linspace(0.8,0.01,len(freqs))

# Feed these arrays instead of constants as previously
results = make_los(parameters, 
                   freqs=freqs, 
                   q_noise_in=q_noise_in, 
                   u_noise_in=u_noise_in, 
                   do_rmsynth=True)

quick_plot(results)


### (d) Frequency-dependent Gaussian random noise, user-defined error estimate arrays for q and u:

In [ ]:
# Make array of q and u standard deviations for frequency-dependent noise
q_noise_in = np.linspace(0.8,0.01,len(freqs))
u_noise_in = np.linspace(0.8,0.01,len(freqs))

# Make frequency-dependent error estimates for q and u
# Note: here, using same values as noise standard deviation, which makes sense 
# if the noise is known, but we can experiment with other estimates
dq_in = np.linspace(0.8,0.01,len(freqs))
du_in = np.linspace(0.8,0.01,len(freqs))

# Feed in both noise and error estimate arrays:
results = make_los(parameters, 
                   freqs=freqs, 
                   q_noise_in=q_noise_in, 
                   u_noise_in=u_noise_in,
                   dq_in=dq_in, 
                   du_in=du_in, 
                   do_rmsynth=True)

quick_plot(results)


## (2) Examples with other Faraday depth configurations
### Note: for all of these, using (low) Gaussian random noise and uniform weights for RM synthesis

### (a) Example with two Faraday depth screens:

In [ ]:
parameters = {
              "pi"  : [  0.5,  0.4],  # PI of each of the two components
              "phi" : [-60.0, 20.0], # Faraday depths of -60 and 20 for the two components
              "psi" : [ 10.0, 40.0], # Polarization angles of 10 degrees and 40 degrees for the two components
              "sig" : [  0.0,  0.0],  # No Burn depolarization
              "dphi": [  0.0,  0.0]   # No Burn slab
            }
freqs = np.arange(400e6,1800e6,0.5e6) # Frequencies covering 400 MHz to 1.8 GHz in steps of 0.5 MHz

results = make_los(parameters, 
                   freqs=freqs, 
                   q_noise_in=0.05, 
                   u_noise_in=0.05, 
                   do_rmsynth=True)

quick_plot(results)


### (b) Example with three Faraday depth screens:

In [ ]:
parameters = {
              "pi"  : [  0.5,  0.4, 0.1],  # PI of each of the three components
              "phi" : [-60.0, 20.0, 40.0], # Faraday depths of -60 and 20 for the three components
              "psi" : [ 10.0, 40.0, 30.0], # Polarization angles of 10, 40, 30 degrees for the three components
              "sig" : [  0.0,  0.0,  0.0],  # No Burn depolarization
              "dphi": [  0.0,  0.0,  0.0]   # No Burn slab
            }
freqs = np.arange(400e6,1800e6,0.5e6) # Frequencies covering 400 MHz to 1.8 GHz in steps of 0.5 MHz

results = make_los(parameters, 
                   freqs=freqs, 
                   q_noise_in=0.05, 
                   u_noise_in=0.05, 
                   do_rmsynth=True)

quick_plot(results)

### (c) Example with Burn slab:

In [ ]:
parameters = {
              "pi"  : [1.0],  # PI of the one component
              "phi" : [50.0], # Faraday depth of 50 (for Burn slab, this defines centre FD)
              "psi" : [10.0], # Polarization angle of 10 degrees
              "sig" : [0.0],  # No Burn depolarization
              "dphi": [100.0] # Burn slab of width 100 rad/m^2
            }
freqs = np.arange(400e6,1800e6,0.5e6) # Frequencies covering 400 MHz to 1.8 GHz in steps of 0.5 MHz

results = make_los(parameters, 
                   freqs=freqs, 
                   q_noise_in=0.05, 
                   u_noise_in=0.05, 
                   do_rmsynth=True)

quick_plot(results)

### (d) Burn slab at DRAGONS frequencies:

In [ ]:
parameters = {
              "pi"  : [1.0],  # PI of the one component
              "phi" : [50.0], # Faraday depth of 50 (for Burn slab, this defines centre FD)
              "psi" : [10.0], # Polarization angle of 10 degrees
              "sig" : [0.0],  # No Burn depolarization
              "dphi": [100.0] # Burn slab of width 100 rad/m^2
            }
freqs = np.arange(350e6,1030e6,1e6)

results = make_los(parameters, 
                   freqs=freqs, 
                   q_noise_in=0.05, 
                   u_noise_in=0.05, 
                   do_rmsynth=True)

quick_plot(results)

### (e) Burn slab at LOFAR frequencies:

In [ ]:
parameters = {
              "pi"  : [1.0],  # PI of the one component
              "phi" : [50.0], # Faraday depth of 50 (for Burn slab, this defines centre FD)
              "psi" : [10.0], # Polarization angle of 10 degrees
              "sig" : [0.0],  # No Burn depolarization
              "dphi": [100.0] # Burn slab of width 100 rad/m^2
            }
freqs = np.arange(120e6,170e6,5e3) 

results = make_los(parameters, 
                   freqs=freqs, 
                   q_noise_in=0.01, 
                   u_noise_in=0.01, 
                   do_rmsynth=True)

quick_plot(results)